In [22]:
import numpy as numpy
import pandas as pd

In [23]:
data = pd.read_csv("data.csv")
data.head()

,review,sentiment
0,is happi dont havework pratt day,positive
1,bounc rush make feel nauseou,negative
2,im soo bore hot littl sister sooo annoy cant s...,negative
3,i love i get pictur everyminut famili key west...,negative
4,parkson secret recip big fat bitch the servic ...,negative


In [24]:
frac, max_features = 0.03, 5000

In [25]:
data = data.sample(frac=frac)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46433 entries, 1098784 to 1366602
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     46260 non-null  object
 1   sentiment  46433 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [27]:
data.sentiment.value_counts()

negative    23380
positive    23053
Name: sentiment, dtype: int64

In [28]:
#check if there are multiple same reviews in the dataset
data.duplicated().value_counts()
#removing those values
data.drop_duplicates(inplace=True)

In [29]:
from sklearn.utils import resample

pos = data[data.sentiment == "positive"]
neg = data[data.sentiment == "negative"]

undersampled_pos = resample(pos, replace = True, n_samples = len(neg), random_state = 42)
data = pd.concat([undersampled_pos, neg])
data = data.dropna()

In [30]:
data.sentiment.value_counts()

positive    23161
negative    23161
Name: sentiment, dtype: int64

In [31]:
#vectorizing the text
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=max_features) #play with max_features=[number] argument

In [32]:
X = cv.fit_transform(data.review).toarray()

In [33]:
X.shape

(46322, 5000)

In [34]:
#encoding the label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data.sentiment = le.fit_transform(data.sentiment)

In [35]:
y = data.iloc[:,-1].values

In [36]:
y.shape

(46322,)

#Builiding the model

In [37]:
#split the data
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.4, random_state=42, stratify=data.sentiment)

In [38]:
#import the navieBayes models
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

clf0, clf1, clf2 = GaussianNB(), MultinomialNB(), BernoulliNB()

clf0.fit(train_X, train_y)
clf1.fit(train_X, train_y)
clf2.fit(train_X, train_y)

BernoulliNB()

In [39]:
pred0 = clf0.predict(test_X)
pred1 = clf1.predict(test_X)
pred2 = clf2.predict(test_X)

In [40]:
#calculating the accuracy of each algo
from sklearn.metrics import accuracy_score

print(f"Gaussian Naive Bayes: {accuracy_score(pred0, test_y)}")
print(f"Multinomial Naive Bayes: {accuracy_score(pred1, test_y)}")
print(f"Bernoulli Navie Bayes: {accuracy_score(pred2, test_y)}")

Gaussian Naive Bayes: 0.5842193318581683
Multinomial Naive Bayes: 0.7418101354633277
Bernoulli Navie Bayes: 0.7445625775810891


In [41]:
#to track a word and its corresponding number
features_dict={}
for i in range(len(cv.get_feature_names_out())):
    features_dict[cv.get_feature_names_out()[i]]=i

In [ ]:
#saving the model to pickle file
import pickle
pickle.dump(data, open("dataframe.pkl", "wb"))
pickle.dump(features_dict, open("features_dict.pkl", "wb"))
